# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = '../WeatherPy/cities_weather.csv'
cities_df = pd.read_csv(cities)
cities_df.dropna(inplace = True) 
cities_df.head(10)




            


,Unnamed: 0,City,Longitude,Latitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Hobart,147.3294,-42.8794,89.01,29,100,16.11,AU,1613717991
1,1,Hilo,-155.0900,19.7297,71.60,78,1,4.61,US,1613717992
2,2,Ribeira Grande,-28.7000,38.5167,57.97,70,100,31.61,PT,1613717992
3,3,Nuevo Progreso,-97.9500,23.1667,46.99,57,40,8.99,MX,1613717993
4,4,Margate,1.3862,51.3813,44.01,84,100,4.00,GB,1613717993
5,5,Busselton,115.3333,-33.6500,75.00,46,0,3.00,AU,1613717994
6,6,Tuktoyaktuk,-133.0374,69.4541,-18.40,76,90,9.22,CA,1613717995
7,7,Manavālakurichi,77.3000,8.1333,87.80,66,20,3.13,IN,1613717995
8,8,Kaseda-shirakame,130.3167,31.4167,51.80,50,20,17.27,JP,1613717996
9,9,Thompson,-97.8558,55.7435,-13.00,84,20,5.39,CA,1613717885


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

# fill NaN values and convert to float
humidity = cities_df['Humidity'].astype(float)
# Store latitude and longitude in locations
locations = cities_df[['Latitude','Longitude']]
#plot heatmap
fig = gmaps.figure()
# creat heat layers
heat_layer = gmaps.heatmap_layer(locations,weights=humidity,dissipating=False, max_intensity=10,point_radius=1)

# Add layer

fig.add_layer(heat_layer)

# Display figure

fig




Figure(layout=FigureLayout(height='420px'))

In [4]:
gmaps.figure()

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
hotel_df= cities_df.loc[(cities_df['Max Temp'] < 80 ) & (cities_df['Max Temp'] > 70) & (cities_df['Cloudiness']== 0) & (cities_df['Wind Speed'] < 10)]\
    .dropna()


hotel_df = hotel_df.drop_duplicates('City',keep='first')
hotel_df


,Unnamed: 0,City,Longitude,Latitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,Busselton,115.3333,-33.6500,75.00,46,0,3.00,AU,1613717994
10,10,Jaqué,-78.1703,7.5175,76.80,83,0,2.62,PA,1613717997
14,14,Pokhara,83.9833,28.2333,74.95,23,0,5.26,NP,1613717709
96,96,Morón,-58.6198,-34.6534,75.99,75,0,1.01,AR,1613718079
230,230,Brits,27.7802,-25.6347,78.01,43,0,4.61,ZA,1613718030
231,231,Abu Dhabi,54.3667,24.4667,77.00,65,0,5.75,AE,1613718020
277,277,Uttarkāshi,78.4500,30.7333,73.40,33,0,4.61,IN,1613718192
315,315,Pilar,-35.9567,-9.5972,71.60,94,0,5.75,BR,1613718215
342,342,Paita,-81.1144,-5.0892,73.15,81,0,5.57,PE,1613718232
347,347,Kruisfontein,24.7314,-34.0033,73.99,58,0,1.99,ZA,1613718236


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df['Hotel Name'] = ''
hotel_df

,Unnamed: 0,City,Longitude,Latitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
5,5,Busselton,115.3333,-33.6500,75.00,46,0,3.00,AU,1613717994,
10,10,Jaqué,-78.1703,7.5175,76.80,83,0,2.62,PA,1613717997,
14,14,Pokhara,83.9833,28.2333,74.95,23,0,5.26,NP,1613717709,
96,96,Morón,-58.6198,-34.6534,75.99,75,0,1.01,AR,1613718079,
230,230,Brits,27.7802,-25.6347,78.01,43,0,4.61,ZA,1613718030,
231,231,Abu Dhabi,54.3667,24.4667,77.00,65,0,5.75,AE,1613718020,
277,277,Uttarkāshi,78.4500,30.7333,73.40,33,0,4.61,IN,1613718192,
315,315,Pilar,-35.9567,-9.5972,71.60,94,0,5.75,BR,1613718215,
342,342,Paita,-81.1144,-5.0892,73.15,81,0,5.57,PE,1613718232,
347,347,Kruisfontein,24.7314,-34.0033,73.99,58,0,1.99,ZA,1613718236,


In [7]:
params = {
    'radius':5000,
    'types':'lodging',
    'key':g_key
}

for index, row in hotel_df.iterrows():
    Lat = row['Latitude']
    Lng = row['Longitude']
    params['location'] =f'{Lat},{Lng}'

    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels = requests.get(url, params=params)
    hotels = hotels.json()


    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except:
        print('data not available...skipped')
hotel_df

data not available...skipped


,Unnamed: 0,City,Longitude,Latitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
5,5,Busselton,115.3333,-33.6500,75.00,46,0,3.00,AU,1613717994,Observatory Guest House
10,10,Jaqué,-78.1703,7.5175,76.80,83,0,2.62,PA,1613717997,Imbu Bua Lodge
14,14,Pokhara,83.9833,28.2333,74.95,23,0,5.26,NP,1613717709,Butterfly Lodge
96,96,Morón,-58.6198,-34.6534,75.99,75,0,1.01,AR,1613718079,Hotel Morón
230,230,Brits,27.7802,-25.6347,78.01,43,0,4.61,ZA,1613718030,By Die Vlei
231,231,Abu Dhabi,54.3667,24.4667,77.00,65,0,5.75,AE,1613718020,Ramee Garden Hotel Apartments
277,277,Uttarkāshi,78.4500,30.7333,73.40,33,0,4.61,IN,1613718192,Anand Ganga
315,315,Pilar,-35.9567,-9.5972,71.60,94,0,5.75,BR,1613718215,Pousada e churrascaria Manguaba
342,342,Paita,-81.1144,-5.0892,73.15,81,0,5.57,PE,1613718232,Hotel Náutico de Paita
347,347,Kruisfontein,24.7314,-34.0033,73.99,58,0,1.99,ZA,1613718236,Oyster Bay House Rental


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [9]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

fig


# Display figure


Figure(layout=FigureLayout(height='420px'))